<a href="https://colab.research.google.com/github/wombat-42/BIS-LAB/blob/main/Parallel_cellular_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random

# Objective Function: Rastrigin Function (minimize)
def rastrigin_function(x, A=10):
    n = len(x)
    return A * n + sum(x_i**2 - A * np.cos(2 * np.pi * x_i) for x_i in x)

# Fitness Function (inverse of the objective function for maximization)
def fitness(x, A=10):
    return -rastrigin_function(x, A)  # Negated because PCA uses maximization rules

# Initialize the grid with random values
def initialize_grid(grid_size, dim, value_range):
    return np.random.uniform(value_range[0], value_range[1], (grid_size, grid_size, dim))

# Get neighbors based on Moore neighborhood
def get_neighbors(grid, x, y):
    neighbors = []
    grid_size = grid.shape[0]
    for i in [-1, 0, 1]:
        for j in [-1, 0, 1]:
            if i == 0 and j == 0:  # Skip the current cell
                continue
            nx, ny = (x + i) % grid_size, (y + j) % grid_size  # Wrap around edges
            neighbors.append(grid[nx, ny])
    return neighbors

# Evolution rule: choose the best neighbor or stay the same
def evolve_cell(grid, x, y):
    current_value = grid[x, y]
    neighbors = get_neighbors(grid, x, y)
    all_candidates = neighbors + [current_value]
    best_value = min(all_candidates, key=lambda candidate: rastrigin_function(candidate))  # Minimize
    return best_value

# Apply mutation to introduce diversity
def mutate(value, mutation_rate, value_range):
    if random.random() < mutation_rate:
        mutation = np.random.uniform(-1, 1, len(value))  # Random mutation in all dimensions
        mutated_value = value + mutation
        return np.clip(mutated_value, value_range[0], value_range[1])  # Keep within bounds
    return value

# Update the entire grid
def update_grid(grid, mutation_rate, value_range):
    new_grid = grid.copy()
    for x in range(grid.shape[0]):
        for y in range(grid.shape[1]):
            evolved_value = evolve_cell(grid, x, y)
            mutated_value = mutate(evolved_value, mutation_rate, value_range)
            new_grid[x, y] = mutated_value
    return new_grid

# Check for convergence
def has_converged(grid, tolerance):
    flat_grid = grid.reshape(-1, grid.shape[2])
    fitness_values = [fitness(val) for val in flat_grid]
    return max(fitness_values) - min(fitness_values) < tolerance

# Check if best solution has repeated
def check_repeated_solution(best_solution, history, iteration, max_repeats=5):  # Relaxed repetition check
    # Allow some iterations before checking for repetition
    if iteration < 5:  # Skip the first 5 iterations for repetition check
        return False
    history.append(best_solution)
    if len(history) > max_repeats:
        history.pop(0)
    # If the last `max_repeats` solutions are identical, stop the algorithm
    return len(set(map(tuple, history))) == 1

# Main function to execute the PCA
def parallel_cellular_algorithm(grid_size, dim, value_range, mutation_rate, max_iterations, tolerance, max_repeats=5):
    grid = initialize_grid(grid_size, dim, value_range)
    iteration = 0
    history = []  # To track previous best solutions
    best_solution = None
    best_fitness = -np.inf  # Maximizing fitness, so start with a very low value

    while iteration < max_iterations:
        grid = update_grid(grid, mutation_rate, value_range)

        # Track progress at each iteration
        flat_grid = grid.reshape(-1, grid.shape[2])
        current_best_solution = min(flat_grid, key=lambda val: rastrigin_function(val))  # Minimize Rastrigin Function
        current_best_fitness = fitness(current_best_solution)

        # If the new solution is better than the previous best
        if current_best_fitness > best_fitness:
            best_solution = current_best_solution
            best_fitness = current_best_fitness

        print(f"Iteration {iteration + 1}: Best Solution = {best_solution}, Fitness = {best_fitness}, Objective = {rastrigin_function(best_solution)}")

        if has_converged(grid, tolerance):
            print("Convergence achieved!")
            break

        # Check if the solution is repeating
        if check_repeated_solution(best_solution, history, iteration, max_repeats):
            print("Solution has repeated multiple times. Stopping early to avoid unnecessary computation.")
            break

        iteration += 1

    # Final grid state and best solution
    return grid, best_solution, best_fitness, iteration

# Run the PCA
if __name__ == "__main__":
    grid_size = 10  # 10x10 grid
    dim = 5  # Dimensionality of the problem
    value_range = (-5.12, 5.12)  # Search space for each dimension (common for Rastrigin)
    mutation_rate = 0.4  # Increased mutation rate for more diversity in solutions
    max_iterations = 100  # Number of iterations to run the algorithm
    tolerance = 5  # Relaxed convergence criterion for more iterations before stopping

    final_grid, best_solution, best_fitness, iterations = parallel_cellular_algorithm(
        grid_size, dim, value_range, mutation_rate, max_iterations, tolerance
    )

    print(f"\nFinal Results:")
    print(f"Best Solution: {best_solution}")
    print(f"Best Fitness: {best_fitness} (Objective Value: {rastrigin_function(best_solution)})")
    print(f"Total Iterations: {iterations}")


Iteration 1: Best Solution = [-1.84820648 -1.89783503 -2.27895477  1.12460484  0.09072402], Fitness = -35.99030935442435, Objective = 35.99030935442435
Iteration 2: Best Solution = [-1.84820648 -1.89783503 -2.27895477  1.12460484  0.09072402], Fitness = -35.99030935442435, Objective = 35.99030935442435
Iteration 3: Best Solution = [-1.84820648 -1.89783503 -2.27895477  1.12460484  0.09072402], Fitness = -35.99030935442435, Objective = 35.99030935442435
Iteration 4: Best Solution = [-1.30170102 -1.00741014 -2.08986584  2.16111587  1.07521069], Fitness = -33.455035936552754, Objective = 33.455035936552754
Iteration 5: Best Solution = [-1.02220941 -1.08008071 -1.87276671  0.98844727  0.97705172], Fitness = -12.145936747663796, Objective = 12.145936747663796
Iteration 6: Best Solution = [-1.02220941 -1.08008071 -1.87276671  0.98844727  0.97705172], Fitness = -12.145936747663796, Objective = 12.145936747663796
Solution has repeated multiple times. Stopping early to avoid unnecessary computat